# SQL Azure Database Basic Monitor - Serverless Aggregation Queries

Basic monitoring queries to get information about your SQL Azure Database performance.

## Prerequisites

1. An Azure Synapse Workspace with a Synapse SQL Serverless Pool
2. Permission provided to the Serverless Pool Managed Identity to access the storage account
3. Data feeding into the storage account

## Initial Setup
These scripts are run to set up your initial views/access to the CSV files. These assume you have the proper permissions set up from your serverless pool to access your blob storage account.

### Credentials
These are the credentials and data source to access the CSV files, make sure to change the location to your storage account and container.
Comment these lines out if you have already created your sql credentials to access the data.

In [ ]:
CREATE MASTER KEY;

CREATE DATABASE SCOPED CREDENTIAL SynapseId
WITH IDENTITY = 'Managed Identity';

CREATE EXTERNAL DATA SOURCE SqlLogging
WITH (    LOCATION   = 'https://[youraccount].blob.core.windows.net/[yourcontainer]/',
          CREDENTIAL = SynapseId
);

### Performance View
This view will give you access to the Sql Performance Data retained from `sys.dm_db_resource_stats`

Make sure to edit the FIELDTERMINATOR to what you used as your LOG_FILE_CSV_SEPARATOR in your function settings.

In [29]:
CREATE OR ALTER VIEW SqlAzurePerformance
AS
SELECT CAST(end_time as DATETIME) as end_time, [avg_cpu_percent], [avg_data_io_percent], [avg_log_write_percent], [avg_memory_usage_percent], [xtp_storage_percent], [max_worker_percent], [max_session_percent], [dtu_limit], [avg_login_rate_percent], [avg_instance_cpu_percent], [avg_instance_memory_percent], [cpu_limit], [replica_role], CAST(perf.filepath(1) as DATE) as [file_date] FROM
    OPENROWSET(
        BULK 'performance-*.csv',
        FORMAT = 'CSV', 
        PARSER_VERSION = '2.0',
        FIELDTERMINATOR ='|',
        DATA_SOURCE = 'SqlLogging',
        ROWSET_OPTIONS = '{"READ_OPTIONS":["ALLOW_INCONSISTENT_READS"]}'
    )
    WITH (
        [end_time] VARCHAR(25) COLLATE Latin1_General_100_BIN2_UTF8,
        [avg_cpu_percent] DECIMAL(8,2),
        [avg_data_io_percent] DECIMAL(8,2),
        [avg_log_write_percent] DECIMAL(8,2),
        [avg_memory_usage_percent] DECIMAL(8,2),
        [xtp_storage_percent] DECIMAL(8,2),
        [max_worker_percent] DECIMAL(8,2),
        [max_session_percent] DECIMAL(8,2),
        [dtu_limit] INT,
        [avg_login_rate_percent] DECIMAL(8,2),
        [avg_instance_cpu_percent] DECIMAL(8,2),
        [avg_instance_memory_percent] DECIMAL(8,2),
        [cpu_limit] INT,
        [replica_role] INT
    ) 
    as [perf]

Commands completed successfully.

Total execution time: 00:00:08.019

## Example Queries
Here are some example queries to start using the performance data. 

When writing your own queries, make sure to timebox your data using `file_date` as the predicate as this will use partition elimination to improve your query performance.

### Top 5 instances of CPU Usage over the last week

In [30]:
SELECT TOP 5 end_time, avg_cpu_percent, avg_data_io_percent, avg_log_write_percent
FROM SqlAzurePerformance WHERE file_date > DATEADD(day, -7, CURRENT_TIMESTAMP) ORDER BY avg_cpu_percent DESC

Statement ID: {B8259492-089C-49B6-8CF0-069678413392} | Query hash: 0xA535616A1B83920E | Distributed request ID: {45F8C328-5BB3-4E84-A566-A5FFEAEC2D96}. Total size of data scanned is 1 megabytes, total size of data moved is 1 megabytes, total size of data written is 0 megabytes.

(5 rows affected)

Total execution time: 00:00:11.128

end_time,avg_cpu_percent,avg_data_io_percent,avg_log_write_percent
2022-10-27 01:39:44.000,18.05,0.00,0.00
2022-10-27 01:42:44.000,7.14,0.00,0.00
2022-10-27 01:42:59.000,7.08,0.00,0.00
2022-10-27 01:44:14.000,6.86,0.00,0.00
2022-10-27 01:42:29.000,6.54,0.00,0.00
